# 1. Condition Variables
## 1.1. Background
Mututexes are great for locking things 'away' (so that **only one thread** can **access** them at a time), but they are not sufficient for more complex synchronization schemes where **a thread is expected to wait for a condition to become true**. 

That is, mutexes are not good for cooperating threads that want simply inform each other about something important. This problem is handled by **condition variables** in the pthreads.

## 1.2. Definition
A **conditional variable** is a kind of synchronization object that a thread can **wait for a condition to become true**. A condition variable is always associated with a **mutex**. The **mutex** is required for a condition variable to work. 

A thread may wait on a **condition variable** only inside of a **critical section** that is protected by the **mutex** associated with the condition variable. 
* Before blocking on a condition variable, a thread must **aquire (lock)** the the associated mutex. 
* When a thread blocks on a condition variable, the associated mutex is **released (unlocked)**. 

When a thread blocked on a **condition variable** is woken up, pthread takes the following action:
* To **wake up** a thread ```t1``` blocked on a condition, a thread ```t2``` must **lock** the associated **mutex** first.
* Then ```t1``` is **unlocked and inserted** to the mutex queue. 
* When ```t2``` will **unlock** the mutex, ```t1``` will eventually compete for it with other threads.

**Note** that When ```t1``` is woken up and lock the mutex again, the **condition** might be **False** again
<img src="resources/ex_cond1.png" alt="Drawing" style="width: 640px;"/>
**The solution** is that ```t``` **must** test the condition again, block if needed

# 2. Condition Variable Usage
## 2.1. Declaring and initializing condition variables
The condition variables are of type ```pthread_cond_t```, and they have attributes ```pthread_condattr_t```. Details are available in pthreads standard.

Before use,  **condition variables** must be initialized using

```c
    int pthread_cond_init(pthread_cond_t *cond, const pthread_condattr_t * cond_attr)
```

After use,  **condition variables** can be destroyed by
```c
    int pthread_cond_destroy(pthread_cond_t *cond)
```

Again, attributes can be used in the ```pthread_cond_init()``` function: 
* to create a **default** condition variable, you can set ```cond_attr``` to NULL, equivalent to ```pthread_cond_t condition_variable = PTHREAD_COND_INITIALIZER;```

## 2.2 Waiting on a condition variable
To wait on a condition variable, the folliwng function should be used:
```c
    int pthread_cond_wait(pthread_cond_t *cond, pthread_mutex_t *mutex)
```
A thread can block on a condition by calling ```pthread_cond_wait()```. 
Note that it must **first lock** the associated mutex: after waking up, the condition **must be checked again**! A ```while()``` loop is required to check the **condition**, s.t.
```c
    pthread_mutex_lock(&m); // Acquire resources
    /* Do something before checking */
    while (!condition) {
        pthread_cond_wait(&cond_var, &m);
    }
    /* Do something after checking */
    pthread_mutex_unlock(&m); // Release resources
```

Similiarly to mutexes, condition variable also supports waits with timeouts. The function is:
```c
    int pthread_cond_timedwait (pthread_cond_t *cond, pthread_mutex_t *mut, const struct timespec *abstime);
```

The function will block first. When ```abstime``` is reached without being unblocked, the function will return with an ```ETIMEDOUT``` error code.

## 2.3. Waking up a thread blocked on a condition variable
A thread can wake up another thread that was blocked on a condition variable with the following operation:
```c
    int pthread_cond_signal(pthread_cond_t *cond);
    int pthread_cond_broadcast(pthread_cond_t *cond);
```
A thread can wake up when:
* **(1)** another thread blocked on a condition. By calling ```pthread_cond_signal()``` 
* or **(2)** all the threads blocked on a condition, by calling ```pthread_cond_broadcast()```

**Note** that the thread waking up the other(s) must **first lock** the associated mutex. If no thread is blocked on ```cond```, nothing happens.

# 3. Prelab
## 3.1. Implement a circular buffer
### Step1: Define "element" in buffer
Define the data type for the element contained in the buffer. (A ```struct``` with two fields, one ```int```, and one ```char```)

### Step2: Define the buffer
Define the data type for the buffer itself. The buffer will have:
* Content array with a fixed number (say 16) elements
* A read pointer and a write pointer
    * ```"ReadPtr"```: An integer index into the **content array** that selects the next element to be read from the buffer
    * ```"WritePtr"```: An integer index into the **content array** that select the place for the next element to be written into the buffer
* A counter: An integer shows how many elements are present in the buffer
* A mutex (to lock the buffer)
* A conditional variable (that threads can wait on)

(A ```struct``` with these fields)

### Step3: Buffer initialization ```buffer_int()```
Define the function that initializes a buffer structure
```c
    void buffer_init(struct buffer*b)
```
* All the indices and counter are set to 0
* and the mutex and the condition variable should be initialized according to the pthreads standard

### Step4: Add element method ```enqueue()```
Define a function for adding elements to the buffer
```c
    void enqueue(struct buffer * b, char name, int value)
```
The last two arguments are values to be place into the **next** available element in the buffer. 

The function should **lock the mutex** then **check the condition** for the buffer being full. 
* If the buffer is full, it has to wait on the condition variable. 
* If the buffer is not full:
    * Add the element to the buffer
    * Increment the write pointer (modulo buffer size) and the counter. 
    * Wake up alll other threads waiting on the condition variable, and then unlock the mutex and return

### Step5: Remove element method ```dequeue()```
It's a mirror to ```enqueue()```
```c
    void dequeue(struct buffer* b, char* name, int* value)
```

The last two arguments are values to be place that will hold the values retrieved from the buffer. 
The function should **lock the mutex** then **check the condition** for the buffer being full. 
* If the buffer is full, it has to wait on the condition variable. 
* If the buffer is not full:
    * Add the element to the buffer
    * Increment the write pointer (modulo buffer size) and the counter. 
    * Wake up alll other threads waiting on the condition variable, and then unlock the mutex and return
    
The function should **lock the mutex** then **check the condition** for the buffer being full. 
* If the buffer is full, it has to wait on the condition variable. 
* If the buffer is not full:
    * Add the element to the buffer
    * Increment the write pointer (modulo buffer size) and the counter. 
    * Wake up alll other threads waiting on the condition variable, and then unlock the mutex and return
    
### Step6: Define a "buffer writer"
Define a **producer** thread function that keeps enqueuing elements into the buffer. 

The function should enqueue a number of, say 1000, elements, Each one having the same name of the producer ```P```and an counter value (```int```).  

The function should print a message that tells the state inside of the function

### Step7: Define a "buffer reader"
Define a **consumer** thread function that keeps dequeuing elements from the buffer.

The function should dequeue a nubmer of, say 1000, elements, and print out the elements's content after each dequeuing operation.

### Step8: Define the ```main()```
The main program should do the following:
* Initialize the buffer
* Launches two threads (One for producer, one for consumer)
* Join the threads and terminates

## 3.2. More Writers
Use the code from **3.1** but create 3 **producer** threads (reuse the thread function ```producer()```you written)

Add testing code that verifies that it has received all data (1000 messages from each producers) and the data should come in the correct order (i.e. increasing values for the message index)

# 4. In the lab
Apartfrom Q1, All in-labs will be basically testing Pre-lab on board. 

Do the prelab plz.